In [1]:
import os, sys, time, glob
import warnings
import re
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import pandana as pdna
pd.options.mode.chained_assignment = None
import h5py
sys.path.append(os.path.join(os.getcwd(),r'..\..\..'))
# from input_configuration import base_year
import toml
config = toml.load(os.path.join(os.getcwd(),r'../../../configuration/input_configuration.toml'))

%matplotlib inline

from IPython.display import display, HTML

if (sys.version_info < (3, 0)):
    HTML('''<script>
    code_show=true; 
    function code_toggle() {
     if (code_show){
     $('div.input').hide();
     } else {
     $('div.input').show();
     }
     code_show = !code_show
    } 
    $( document ).ready(code_toggle);
    </script>
    <form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')


In [2]:
CSS = """
.output {
    flex-direction: row;
}
"""

HTML('<style>{}</style>'.format(CSS))

# Relative path between notebooks and goruped output directories
relative_path = '../../../outputs'
survey_path = '../../../inputs/base_year/survey'

# Write out CSVs with a numeric value prefix to help organize outputs 
incr = 1

In [3]:
pd.options.display.float_format = '{:0,.0f}'.format

In [4]:
def load_network_summary(filepath):
    """Load network-level results using a standard procedure. """
    df = pd.read_csv(filepath)

    # Congested network components by time of day
    df.columns

    # Get freeflow from 20to5 period

    # Exclude trips taken on non-designated facilities (facility_type == 0)
    # These are artificial (weave lanes to connect HOV) or for non-auto uses 
    df = df[df['data3'] != 0]    # data3 represents facility_type

    # calculate total link VMT and VHT
    df['VMT'] = df['@tveh']*df['length']
    df['VHT'] = df['@tveh']*df['auto_time']/60

    # Define facility type
    df.loc[df['data3'].isin([1,2]), 'facility_type'] = 'highway'
    df.loc[df['data3'].isin([3,4,6]), 'facility_type'] = 'arterial'
    df.loc[df['data3'].isin([5]), 'facility_type'] = 'connector'

    # Calculate delay
    # Select links from overnight time of day
    delay_df = df.loc[df['tod'] == '20to5'][['ij','auto_time']]
    delay_df.rename(columns={'auto_time':'freeflow_time'}, inplace=True)

    # Merge delay field back onto network link df
    df = pd.merge(df, delay_df, on='ij', how='left')

    # Calcualte hourly delay
    df['total_delay'] = ((df['auto_time']-df['freeflow_time'])*df['@tveh'])/60    # sum of (volume)*(travtime diff from freeflow)

    df['county'] =df['@countyid'].map({33: 'King',
                                      35: 'Kitsap',
                                      53: 'Pierce',
                                      61: 'Snohomish'})
    
    return df

In [5]:
output_dir = r'..\..\..\outputs\compare\RTP\person'

# Trips per Day by Resident
County, Center, Regional Geography

Youth, elderly, LEP, Race & Poverty for Region Work/Non-Work

In [6]:
pd.options.display.float_format = '{:0,.1f}'.format
df_trip = pd.read_csv(r'..\..\..\outputs\agg\dash\person_trips.csv')
df_person = pd.read_csv(r'..\..\..\outputs\agg\dash\person_geog.csv')

df_trip['is_rgc'] = 0
df_trip.loc[df_trip['hh_rgc'] != 'Not in RGC', 'is_rgc'] = 1
df_person['is_rgc'] = 0
df_person.loc[df_person['hh_rgc'] != 'Not in RGC', 'is_rgc'] = 1

### County and Region

In [7]:
def trips_per_day(geog): 
    df1 = df_person.groupby(geog).sum()[['psexpfac']]
    df2 = df_trip.groupby(geog).sum()[['trexpfac']]
    df = df1.merge(df2, left_index=True, right_index=True)
    df.loc['Region', 'psexpfac'] = df_person.sum()['psexpfac']
    df.loc['Region', 'trexpfac'] = df_trip.sum()['trexpfac']
    df['Total Trips per Day'] = df['trexpfac']/df['psexpfac']

    df_trip.loc[df_trip['dpurp'] != 'Work', 'trip_type'] = 'Non-Work'
    df_trip.loc[df_trip['dpurp'] == 'Work', 'trip_type'] = 'Work'
    for purp in ['Work','Non-Work']:
        _df = df_trip[df_trip['trip_type'] == purp].groupby(geog).sum()[['trexpfac']]
        _df.rename(columns={'trexpfac': purp + ' Trips'}, inplace=True)

        # Add regional total to bottom of table
        _df.loc['Region', purp + ' Trips'] = df_trip[df_trip['trip_type'] == purp].sum()['trexpfac']
        df = df.merge(_df, left_index=True, right_index=True)

    df['Work Trips per Day'] = df['Work Trips']/df['psexpfac']
    df['Non-Work Trips per Day'] = df['Non-Work Trips']/df['psexpfac']
    df[['Work Trips per Day','Non-Work Trips per Day','Total Trips per Day']]
    
    return df[['Work Trips per Day', 'Non-Work Trips per Day','Total Trips per Day']]

In [8]:
df = trips_per_day('hh_county')
df.to_csv(os.path.join(output_dir,str(incr)+'_trips_per_day_county.csv')); incr+=1
df

,Work Trips per Day,Non-Work Trips per Day,Total Trips per Day
hh_county,,,
King,0.7,3.5,4.2
Kitsap,0.5,3.4,3.9
Pierce,0.5,3.3,3.8
Snohomish,0.6,3.4,4.0
Region,0.6,3.4,4.0


### Regional Growth Centers

In [9]:
df = trips_per_day('is_rgc')
df.index = ['Not in RGC', 'In RGC','Region']
df

,Work Trips per Day,Non-Work Trips per Day,Total Trips per Day
Not in RGC,0.6,3.4,4.0
In RGC,0.9,3.7,4.6
Region,0.6,3.4,4.0


In [10]:
df = trips_per_day('hh_rgc')
df.to_csv(os.path.join(output_dir,str(incr)+'_trips_per_day_rgc.csv')); incr+=1
df

,Work Trips per Day,Non-Work Trips per Day,Total Trips per Day
hh_rgc,,,
Auburn,0.6,3.5,4.0
Ballard-Interbay,0.9,3.7,4.6
Bellevue,1.0,3.7,4.6
Bothell Canyon Park,0.9,3.3,4.2
Bremerton,0.6,3.5,4.1
Burien,0.7,3.4,4.1
Cascade,0.6,3.2,3.8
Duwamish,0.8,3.5,4.3
Everett,0.6,3.5,4.2


### Regional Geographies

In [11]:
df = trips_per_day('hh_rg_proposed')
df.to_csv(os.path.join(output_dir,str(incr)+'_trips_per_day_reg_geog.csv')); incr+=1
df

,Work Trips per Day,Non-Work Trips per Day,Total Trips per Day
hh_rg_proposed,,,
CitiesTowns,0.5,3.4,3.9
Core,0.6,3.4,4.0
HCT,0.6,3.4,4.0
Metro,0.7,3.6,4.3
Rural,0.5,3.2,3.7
UU,0.5,3.3,3.9
Region,0.6,3.4,4.0


### Equity Geographies

In [12]:
# Summary by equity geographies
# Calculate person totals
results_df = pd.DataFrame()
trip_rate_df = pd.DataFrame()
list_50 = ['hh_racial','hh_poverty']

equity_geogs = ['hh_youth','hh_elderly','hh_english','hh_racial','hh_poverty','hh_disability']
for equity_geog in equity_geogs:
    for geog_type in ['_reg','_50']:
        val = df_person[df_person[equity_geog+geog_type] == 1].sum()['psexpfac']
        results_df.loc[equity_geog+geog_type,'psexpfac'] = val
        
# Calculate total work trips per each group
work_results_df = pd.DataFrame()
equity_geogs = ['hh_youth','hh_elderly','hh_english','hh_racial','hh_poverty','hh_disability']
for equity_geog in equity_geogs:
    for geog_type in ['_reg','_50']:
        for purp in ['Non-Work','Work']:
            _df = df_trip[df_trip['trip_type'] == purp].groupby(equity_geog+geog_type).sum()[['trexpfac']]
            if 1 in _df.index:
                if geog_type == '_reg' or equity_geog in list_50:
                    _rate_df = pd.DataFrame([purp,
                                         equity_geog+geog_type,
                                         _df.loc[1].trexpfac/results_df.loc[equity_geog+geog_type].psexpfac]).T
                trip_rate_df = trip_rate_df.append(_rate_df) 
trip_rate_df.columns = ['Trip Type','Equity Group','Trip Rate']
df = trip_rate_df.pivot_table(columns='Trip Type', index='Equity Group', values='Trip Rate', aggfunc='sum')
df['Total Trips per Day'] = df['Non-Work'] + df['Work']
df.rename(columns={'Non-Work': 'Non-Work Trips per Day', 'Work': 'Work Trips per Day'}, inplace=True)
df = df.reset_index()
df = df.rename_axis(None, axis=1)
df.to_csv(os.path.join(output_dir,str(incr)+'_trips_per_day_equity.csv'))
HTML(df[['Equity Group','Work Trips per Day','Non-Work Trips per Day','Total Trips per Day']].to_html(index=False))

Equity Group,Work Trips per Day,Non-Work Trips per Day,Total Trips per Day
hh_disability_reg,0.6,3.4,3.9
hh_elderly_reg,1.7,3.4,5.1
hh_english_reg,0.6,3.5,4.1
hh_poverty_50,0.5,3.4,3.9
hh_poverty_reg,0.6,3.4,4.0
hh_racial_50,0.6,3.5,4.1
hh_racial_reg,0.6,3.5,4.1
hh_youth_reg,0.6,3.4,4.0


In [13]:
# Summary by equity geographies
# Calculate person totals
results_df = pd.DataFrame()
trip_rate_df = pd.DataFrame()
list_50 = ['hh_racial','hh_poverty']

equity_geogs = ['hh_youth','hh_elderly','hh_english','hh_racial','hh_poverty','hh_disability']
for equity_geog in equity_geogs:
    for geog_type in ['_reg','_50']:
        val = df_person[df_person[equity_geog+geog_type] == 0].sum()['psexpfac']
        results_df.loc[equity_geog+geog_type,'psexpfac'] = val
        
# Calculate total work trips per each group
work_results_df = pd.DataFrame()
equity_geogs = ['hh_youth','hh_elderly','hh_english','hh_racial','hh_poverty','hh_disability']
for equity_geog in equity_geogs:
    for geog_type in ['_reg','_50']:
        for purp in ['Non-Work','Work']:
            _df = df_trip[df_trip['trip_type'] == purp].groupby(equity_geog+geog_type).sum()[['trexpfac']]
            if 1 in _df.index:
                if geog_type == '_reg' or equity_geog in list_50:
                    _rate_df = pd.DataFrame([purp,
                                         equity_geog+geog_type,
                                         _df.loc[1].trexpfac/results_df.loc[equity_geog+geog_type].psexpfac]).T
                trip_rate_df = trip_rate_df.append(_rate_df) 
trip_rate_df.columns = ['Trip Type','Equity Group','Trip Rate']
df = trip_rate_df.pivot_table(columns='Trip Type', index='Equity Group', values='Trip Rate', aggfunc='sum')
df['Total Trips per Day'] = df['Non-Work'] + df['Work']

df.rename(columns={'Non-Work': 'Non-Work Trips per Day', 'Work': 'Work Trips per Day'}, inplace=True)
df = df.reset_index()
df = df.rename_axis(None, axis=1)
df['Equity Group'] = ['NOT in '+i for i in df['Equity Group']]
df.to_csv(os.path.join(output_dir,str(incr)+'_trips_per_day_equity.csv'))
HTML(df[['Equity Group','Work Trips per Day','Non-Work Trips per Day','Total Trips per Day']].to_html(index=False))

Equity Group,Work Trips per Day,Non-Work Trips per Day,Total Trips per Day
NOT in hh_disability_reg,0.5,2.9,3.3
NOT in hh_elderly_reg,1.5,2.9,4.4
NOT in hh_english_reg,0.4,2.2,2.5
NOT in hh_poverty_50,0.0,0.1,0.1
NOT in hh_poverty_reg,0.4,2.4,2.8
NOT in hh_racial_50,0.2,0.9,1.1
NOT in hh_racial_reg,0.5,2.8,3.3
NOT in hh_youth_reg,0.7,4.0,4.7


# Miles Driven per Day by Resident
County, Center, Regional Geography

Youth, elderly, LEP, Race & Poverty for Region Work/Non-Work

In [14]:
pd.options.display.float_format = '{:0,.1f}'.format
df_vmt = pd.read_csv(r'..\..\..\outputs\agg\dash\person_vmt.csv')
df_person = pd.read_csv(r'..\..\..\outputs\agg\dash\person_geog.csv')

df_vmt['is_rgc'] = 0
df_vmt.loc[df_vmt['hh_rgc'] != 'Not in RGC', 'is_rgc'] = 1
df_person['is_rgc'] = 0
df_person.loc[df_person['hh_rgc'] != 'Not in RGC', 'is_rgc'] = 1

# Select only drivers (dorp = 1) and auto trips
df_vmt = df_vmt[df_vmt['mode'].isin(['SOV','HOV2','HOV3+']) & (df_vmt['dorp'] == 1)]

def vmt_per_person(df_vmt, df_person, geog):
    _df_vmt = df_vmt.groupby(geog).sum()[['travdist_wt']]
    _df_person = df_person.groupby(geog).sum()[['psexpfac']]

    df = _df_vmt.merge(_df_person, left_index=True, right_index=True)
    df.loc['Region',:] = df.sum(axis=0)
    df['Average Miles per Person'] = df['travdist_wt']/df['psexpfac']
    
    return df[['Average Miles per Person']]

### County

In [15]:
df = vmt_per_person(df_vmt, df_person, 'hh_county')
df.to_csv(os.path.join(output_dir,str(incr)+'_vmt_county.csv')); incr+=1
df

,Average Miles per Person
hh_county,
King,16.0
Kitsap,15.5
Pierce,17.9
Snohomish,19.8
Region,17.1


### Regional Growth Centers

In [16]:
df = vmt_per_person(df_vmt, df_person, 'is_rgc')
df.index = ['Not in RGC', 'In RGC','Region']
df

,Average Miles per Person
Not in RGC,17.7
In RGC,7.2
Region,17.1


In [17]:
df = vmt_per_person(df_vmt, df_person, 'hh_rgc')
df.to_csv(os.path.join(output_dir,str(incr)+'_vmt_rgc.csv')); incr+=1
df

,Average Miles per Person
hh_rgc,
Auburn,12.8
Ballard-Interbay,9.3
Bellevue,6.9
Bothell Canyon Park,20.7
Bremerton,5.7
Burien,14.0
Cascade,17.9
Duwamish,10.8
Everett,8.6


### Regional Geography

In [18]:
df = vmt_per_person(df_vmt, df_person, 'hh_rg_proposed')
df.to_csv(os.path.join(output_dir,str(incr)+'_vmt_reg_geog.csv')); incr+=1
df

,Average Miles per Person
hh_rg_proposed,
CitiesTowns,21.8
Core,16.4
HCT,17.5
Metro,11.6
Rural,26.3
UU,19.5
Region,17.1


### Equity Geographies
Average Miles per Person (by residence inside or outside equity geography) 

In [19]:
result_df = pd.DataFrame()
list_50 = ['hh_racial','hh_poverty']
equity_geogs = ['hh_youth','hh_elderly','hh_english','hh_racial','hh_poverty','hh_disability']
for equity_geog in equity_geogs:
    for geog_type in ['_reg','_50']:
        if geog_type == '_reg' or equity_geog in list_50:
            col = equity_geog+geog_type
            df = vmt_per_person(df_vmt, df_person, col)
            if 1 in df.index:
                df = df.loc[1]
            df['source'] = col
            result_df = result_df.append(df)
result_df.index = result_df.source
result_df.drop('source', axis=1, inplace=True)
result_df = result_df[~result_df.index.duplicated()]
df_in = result_df.copy()

result_df = pd.DataFrame()
list_50 = ['hh_racial','hh_poverty']
equity_geogs = ['hh_youth','hh_elderly','hh_english','hh_racial','hh_poverty','hh_disability']
for equity_geog in equity_geogs:
    for geog_type in ['_reg','_50']:
        if geog_type == '_reg' or equity_geog in list_50:
            col = equity_geog+geog_type
            df = vmt_per_person(df_vmt, df_person, col)
            if 0 in df.index:
                df = df.loc[0]
            df['source'] = col
            result_df = result_df.append(df)
result_df.index = result_df.source
result_df.drop('source', axis=1, inplace=True)
result_df = result_df[~result_df.index.duplicated()]
# result_df.index = ['NOT in '+i for i in result_df.index]


df_out = result_df.copy()

df = df_in.merge(df_out, left_index=True, right_index=True)
df.columns = ['Inside Equity Geography',  'Outside Equity Geography']
df.to_csv(os.path.join(output_dir,str(incr)+'_vmt_equity.csv')); incr+=1
df

,Inside Equity Geography,Outside Equity Geography
source,,
hh_youth_reg,18.8,15.1
hh_elderly_reg,17.7,16.6
hh_english_reg,15.7,18.0
hh_racial_reg,15.2,18.7
hh_racial_50,14.8,17.7
hh_poverty_reg,15.3,18.4
hh_poverty_50,11.7,17.2
hh_disability_reg,17.1,17.2


# Miles Walking and Biking per Day by Resident
County, Center, Regional Geography

Youth, elderly, LEP, Race & Poverty for Region Work/Non-Work

In [20]:
df_vht = pd.read_csv(r'..\..\..\outputs\agg\dash\person_vht.csv')
df_hh = pd.read_csv(r'..\..\..\outputs\agg\dash\hh_geog.csv')



def walk_bike_per_person_time(df_vmt, df_person, geog):
    
    results_df = pd.DataFrame()
    for mode in ['Walk','Bike']:
        
        _df_vmt = df_vmt[df_vmt['mode'] == mode]
        _df_vmt = _df_vmt.groupby(geog).sum()[['travtime_wt']]
        _df_hh = df_hh.groupby(geog).sum()[['hhexpfac']]

        df = _df_vmt.merge(_df_hh, left_index=True, right_index=True)
        df.loc['Region',:] = df.sum(axis=0)
        df['Average Miles per Person'] = df['travtime_wt']/df['hhexpfac']
        results_df[mode] = df['Average Miles per Person']
    
    _df_vmt = df_vmt[df_vmt['mode'].isin(['Walk','Bike'])]
    _df_vmt = _df_vmt.groupby(geog).sum()[['travtime_wt']]
    _df_hh = df_hh.groupby(geog).sum()[['hhexpfac']]

    df = _df_vmt.merge(_df_hh, left_index=True, right_index=True)
    df.loc['Region',:] = df.sum(axis=0)
    df['Average Miles per Person'] = df['travtime_wt']/df['hhexpfac']
    results_df['Walk and Bike'] = df['Average Miles per Person']
    
    return results_df

df = walk_bike_per_person_time(df_vht, df_hh, 'hh_county')
# df.to_csv(os.path.join(output_dir,str(incr)+'_mile_bikewalk_county.csv')); incr+=1
df

,Walk,Bike,Walk and Bike
hh_county,,,
King,26.5,3.5,30.0
Kitsap,22.3,2.8,25.1
Pierce,21.7,3.6,25.2
Snohomish,21.4,3.3,24.8
Region,24.4,3.4,27.8


In [21]:
# df = pd.read_csv(r'..\..\..\outputs\agg\dash\trip_distance.csv')
# _df = df[df['mode'] == 'Bike']
# (_df['travdist_bin']*_df['trexpfac']).sum()/_df['trexpfac'].sum()

In [22]:
# _df = df[df['mode'] == 'Walk']
# (_df['travdist_bin']*_df['trexpfac']).sum()/_df['trexpfac'].sum()

In [23]:
df_vmt = pd.read_csv(r'..\..\..\outputs\agg\dash\person_vmt.csv')
df_person = pd.read_csv(r'..\..\..\outputs\agg\dash\person_geog.csv')

df_vmt['is_rgc'] = 0
df_vmt.loc[df_vmt['hh_rgc'] != 'Not in RGC', 'is_rgc'] = 1
df_person['is_rgc'] = 0
df_person.loc[df_person['hh_rgc'] != 'Not in RGC', 'is_rgc'] = 1


def walk_bike_per_person(df_vmt, df_person, geog):
    
    results_df = pd.DataFrame()
    for mode in ['Walk','Bike']:
        
        _df_vmt = df_vmt[df_vmt['mode'] == mode]
        _df_vmt = _df_vmt.groupby(geog).sum()[['travdist_wt']]
        _df_person = df_person.groupby(geog).sum()[['psexpfac']]

        df = _df_vmt.merge(_df_person, left_index=True, right_index=True)
        df.loc['Region',:] = df.sum(axis=0)
        df['Average Miles per Person'] = df['travdist_wt']/df['psexpfac']
        results_df[mode] = df['Average Miles per Person']
    
    _df_vmt = df_vmt[df_vmt['mode'].isin(['Walk','Bike'])]
    _df_vmt = _df_vmt.groupby(geog).sum()[['travdist_wt']]
    _df_person = df_person.groupby(geog).sum()[['psexpfac']]

    df = _df_vmt.merge(_df_person, left_index=True, right_index=True)
    df.loc['Region',:] = df.sum(axis=0)
    df['Average Miles per Person'] = df['travdist_wt']/df['psexpfac']
    results_df['Walk and Bike'] = df['Average Miles per Person']
    
    return results_df

### County

In [24]:
df = walk_bike_per_person(df_vmt, df_person, 'hh_county')
df.to_csv(os.path.join(output_dir,str(incr)+'_mile_bikewalk_county.csv')); incr+=1
df

,Walk,Bike,Walk and Bike
hh_county,,,
King,0.5,0.2,0.8
Kitsap,0.4,0.2,0.6
Pierce,0.4,0.2,0.6
Snohomish,0.4,0.2,0.6
Region,0.5,0.2,0.7


### Regional Growth Center 

In [25]:
df = walk_bike_per_person(df_vmt, df_person, 'is_rgc')
df.index = ['Not in RGC', 'Inside RGC', 'Region']
df

,Walk,Bike,Walk and Bike
Not in RGC,0.4,0.2,0.6
Inside RGC,1.1,0.3,1.4
Region,0.5,0.2,0.7


In [26]:
df = walk_bike_per_person(df_vmt, df_person, 'hh_rgc')
df.to_csv(os.path.join(output_dir,str(incr)+'_mile_bikewalk_rgc.csv'))
df

,Walk,Bike,Walk and Bike
hh_rgc,,,
Auburn,0.6,0.4,1.0
Ballard-Interbay,0.8,0.4,1.2
Bellevue,1.0,0.1,1.1
Bothell Canyon Park,0.5,0.4,0.8
Bremerton,0.8,0.2,1.0
Burien,0.5,0.3,0.8
Cascade,0.4,0.2,0.7
Duwamish,0.8,0.3,1.1
Everett,0.8,0.2,1.0


### Regional Geography

In [27]:
df = walk_bike_per_person(df_vmt, df_person, 'hh_rg_proposed')
df.to_csv(os.path.join(output_dir,str(incr)+'_mile_bikewalk_reg_geog.csv')); incr+=1
df

,Walk,Bike,Walk and Bike
hh_rg_proposed,,,
CitiesTowns,0.4,0.2,0.5
Core,0.4,0.2,0.6
HCT,0.4,0.2,0.6
Metro,0.7,0.2,0.9
Rural,0.3,0.2,0.4
UU,0.4,0.2,0.5
Region,0.5,0.2,0.7


### Equity Geographies

In [28]:
result_df = pd.DataFrame()
equity_geogs = ['hh_youth','hh_elderly','hh_english','hh_racial','hh_poverty','hh_disability']
for equity_geog in equity_geogs:
    for geog_type in ['_reg','_50']:
        if geog_type == '_reg' or equity_geog in list_50:
            col = equity_geog+geog_type
            df = walk_bike_per_person(df_vmt, df_person, col)
            if 1 in df.index:
                df = df.loc[1]
            df['source'] = col
            result_df = result_df.append(df)
result_df.index = result_df.source
result_df.drop('source', axis=1, inplace=True)
result_df.to_csv(os.path.join(output_dir,str(incr)+'_mile_bikewalk_equity.csv')); incr+=1
result_df

,Bike,Walk,Walk and Bike
source,,,
hh_youth_reg,0.2,0.4,0.6
hh_elderly_reg,0.2,0.5,0.6
hh_english_reg,0.2,0.5,0.7
hh_racial_reg,0.2,0.5,0.7
hh_racial_50,0.2,0.5,0.7
hh_poverty_reg,0.2,0.5,0.7
hh_poverty_50,0.3,0.6,0.9
hh_disability_reg,0.2,0.5,0.7


In [29]:
result_df = pd.DataFrame()
equity_geogs = ['hh_youth','hh_elderly','hh_english','hh_racial','hh_poverty','hh_disability']
for equity_geog in equity_geogs:
    for geog_type in ['_reg','_50']:
        if geog_type == '_reg' or equity_geog in list_50:
            col = equity_geog+geog_type
            df = walk_bike_per_person(df_vmt, df_person, col)
            if 0 in df.index:
                df = df.loc[0]
            df['source'] = col
            result_df = result_df.append(df)
result_df.index = result_df.source
result_df.drop('source', axis=1, inplace=True)
result_df.index = ['NOT in '+i for i in result_df.index]
result_df.to_csv(os.path.join(output_dir,str(incr)+'_mile_bikewalk_equity.csv')); incr+=1
result_df

,Bike,Walk,Walk and Bike
NOT in hh_youth_reg,0.2,0.6,0.8
NOT in hh_elderly_reg,0.2,0.5,0.7
NOT in hh_english_reg,0.2,0.5,0.7
NOT in hh_racial_reg,0.2,0.5,0.6
NOT in hh_racial_50,0.2,0.5,0.7
NOT in hh_poverty_reg,0.2,0.5,0.6
NOT in hh_poverty_50,0.2,0.5,0.7
NOT in hh_disability_reg,0.2,0.5,0.7


# Jobs Accessible within 45 Minutes of Transit

In [30]:
pd.options.display.float_format = '{:0,.0f}'.format
df = pd.read_csv(r'..\..\..\outputs\access\transit_jobs_access.csv'); incr+=1
df_lu = pd.read_csv(r'..\..\..\inputs\scenario\landuse\parcels_urbansim.txt',
                   delim_whitespace=True, usecols=['PARCELID','EMPTOT_P'])
tot_jobs = df_lu['EMPTOT_P'].sum()

### Region

In [31]:
_df = df[df['geography_group'] == 'region']
_df.index = _df.geography
_df = _df[['value']]
_df.rename(columns={'value': 'Jobs within 45-minute Transit Commute'}, inplace=True)
_df.to_csv(os.path.join(output_dir,str(incr)+'_transit_jobs_45min_region.csv')); incr+=1
_df['% Total Jobs'] = _df['Jobs within 45-minute Transit Commute']/tot_jobs
output = _df.to_html(formatters={
    '% Total Jobs': '{:0,.1%}'.format
}, index=False)
display(HTML(output))

Jobs within 45-minute Transit Commute,% Total Jobs
"133,633",6.6%


### County

In [32]:
_df = df[df['geography_group'] == 'CountyName']
_df.index = _df.geography
_df = _df[['value']]
_df.rename(columns={'value': 'Jobs within 45-minute Transit Commute'}, inplace=True)
_df.to_csv(os.path.join(output_dir,str(incr)+'_transit_jobs_45min_county.csv')); incr+=1
_df['% Total Jobs'] = _df['Jobs within 45-minute Transit Commute']/tot_jobs
output = _df.to_html(formatters={
    '% Total Jobs': '{:0,.1%}'.format
})
display(HTML(output))

,Jobs within 45-minute Transit Commute,% Total Jobs
geography,,
King,"225,124",11.2%
Kitsap,"4,229",0.2%
Pierce,"21,255",1.1%
Snohomish,"27,282",1.4%


### Regional Growth Center

In [33]:
_df = df[df['geography_group'] == 'GrowthCenterName']
_df.index = _df.geography
_df = _df[['value']]
_df.rename(columns={'value': 'Jobs within 45-minute Transit Commute'}, inplace=True)
_df.to_csv(os.path.join(output_dir,str(incr)+'_transit_jobs_45min_rgc.csv')); incr+=1
_df['% Total Jobs'] = _df['Jobs within 45-minute Transit Commute']/tot_jobs
output = _df.to_html(formatters={
    '% Total Jobs': '{:0,.1%}'.format
})
display(HTML(output))

,Jobs within 45-minute Transit Commute,% Total Jobs
geography,,
Auburn,"59,092",2.9%
Ballard-Interbay,"514,833",25.6%
Bellevue,"471,878",23.4%
Bothell Canyon Park,"40,364",2.0%
Bremerton,"35,732",1.8%
Burien,"156,920",7.8%
Cascade,"6,107",0.3%
Duwamish,"482,741",24.0%
Everett,"67,757",3.4%


### Regional Geography

In [34]:
_df = df[df['geography_group'] == 'rg_proposed']
_df.index = _df.geography
_df = _df[['value']]
_df.rename(columns={'value': 'Jobs within 45-minute Transit Commute'}, inplace=True)
_df.to_csv(os.path.join(output_dir,str(incr)+'_transit_jobs_45min_reg_geog.csv')); incr+=1
_df['% Total Jobs'] = _df['Jobs within 45-minute Transit Commute']/tot_jobs
output = _df.to_html(formatters={
    '% Total Jobs': '{:0,.1%}'.format
})
display(HTML(output))

,Jobs within 45-minute Transit Commute,% Total Jobs
geography,,
CitiesTowns,"14,621",0.7%
Core,"55,633",2.8%
HCT,"37,226",1.8%
Metro,"331,604",16.5%
Rural,387,0.0%
UU,"7,976",0.4%


### Equity Geographies

In [35]:
geo_list = []
equity_geogs = ['youth','elderly','english','racial','poverty','disability']
for equity_geog in equity_geogs:
    for geog_type in ['_geog_vs_reg_total','_geog_vs_50_percent']:
        if geog_type == '_geog_vs_reg_total' or equity_geog in ['poverty','racial']:
            geo_list.append(equity_geog+geog_type)

In [36]:
result_df = pd.DataFrame()
for col in geo_list:
    _df = df[(df['geography_group'] == col) & (df['geography'] == '1')]
    result_df = result_df.append(_df[['geography_group', 'value']])
result_df.to_csv(os.path.join(output_dir,str(incr)+'_transit_jobs_45min_equity.csv')); incr+=1
result_df.rename(columns={'geography_group': 'Equity Group', 'value': 'Jobs within 45-minute Transit Commute'}, inplace=True)
# result_df = result_df['']
result_df['% Total Jobs'] = result_df['Jobs within 45-minute Transit Commute']/tot_jobs
output = result_df.to_html(formatters={
    '% Total Jobs': '{:0,.1%}'.format
}, index=False)
display(HTML(output))

Equity Group,Jobs within 45-minute Transit Commute,% Total Jobs
youth_geog_vs_reg_total,"48,612",2.4%
elderly_geog_vs_reg_total,"106,048",5.3%
english_geog_vs_reg_total,"121,402",6.0%
racial_geog_vs_reg_total,"145,713",7.2%
racial_geog_vs_50_percent,"143,411",7.1%
poverty_geog_vs_reg_total,"130,969",6.5%
poverty_geog_vs_50_percent,"216,760",10.8%
disability_geog_vs_reg_total,"95,290",4.7%


In [37]:
result_df = pd.DataFrame()
for col in geo_list:
    _df = df[(df['geography_group'] == col) & (df['geography'] == '0')]
    result_df = result_df.append(_df[['geography_group', 'value']])
result_df.to_csv(os.path.join(output_dir,str(incr)+'_transit_jobs_45min_equity.csv')); incr+=1
result_df.rename(columns={'geography_group': 'Equity Group', 'value': 'Jobs within 45-minute Transit Commute'}, inplace=True)
# result_df = result_df['']
result_df['% Total Jobs'] = result_df['Jobs within 45-minute Transit Commute']/tot_jobs
result_df['Equity Group'] = ['NOT in '+i for i in result_df['Equity Group']]
output = result_df.to_html(formatters={
    '% Total Jobs': '{:0,.1%}'.format
}, index=False)
display(HTML(output))

Equity Group,Jobs within 45-minute Transit Commute,% Total Jobs
NOT in youth_geog_vs_reg_total,"217,198",10.8%
NOT in elderly_geog_vs_reg_total,"158,277",7.9%
NOT in english_geog_vs_reg_total,"140,925",7.0%
NOT in racial_geog_vs_reg_total,"123,851",6.1%
NOT in racial_geog_vs_50_percent,"131,154",6.5%
NOT in poverty_geog_vs_reg_total,"135,565",6.7%
NOT in poverty_geog_vs_50_percent,"131,620",6.5%
NOT in disability_geog_vs_reg_total,"166,919",8.3%


# Average Jobs Accessible within 1 Mile Walk and 3 Mile Bike
Note that this is not using the bike network, but is instead using the all-streets network.
Average accessible jobs are weighted averages based on parcel household population.

In [38]:
df = pd.read_csv(r'..\..\..\outputs\access\walk_bike_jobs_access.csv')

### Regional Average

In [39]:
_df = df[df['geography_group'] == 'region']
_df.index = _df.geography_value
_df = _df[['jobs_1_mile_walk','jobs_3_mile_bike']]
_df.rename(columns={'jobs_1_mile_walk': 'Jobs within 1-mile Walk',
                   'jobs_3_mile_bike': 'Jobs within 3-mile Bike'}, inplace=True)
_df['% Total Jobs (1-mile walk)'] = _df['Jobs within 1-mile Walk']/tot_jobs
_df['% Total Jobs (3-mile bike)'] = _df['Jobs within 3-mile Bike']/tot_jobs
_df.to_csv(os.path.join(output_dir,str(incr)+'_walk_bike_jobs_region.csv')); incr+=1
output = _df.to_html(formatters={
    '% Total Jobs (1-mile walk)': '{:0,.1%}'.format,
    '% Total Jobs (3-mile bike)': '{:0,.1%}'.format,
}, index=False)
display(HTML(output))

Jobs within 1-mile Walk,Jobs within 3-mile Bike,% Total Jobs (1-mile walk),% Total Jobs (3-mile bike)
"9,551","50,557",0.5%,2.5%


### Home County

In [40]:
_df = df[df['geography_group'] == 'CountyName']
_df.index = _df.geography_value
_df = _df[['jobs_1_mile_walk','jobs_3_mile_bike']]
_df.rename(columns={'jobs_1_mile_walk': 'Jobs within 1-mile Walk',
                   'jobs_3_mile_bike': 'Jobs within 3-mile Bike'}, inplace=True)
_df.to_csv(os.path.join(output_dir,str(incr)+'_walk_bike_jobs_county.csv')); incr+=1
_df['% Total Jobs (1-mile walk)'] = _df['Jobs within 1-mile Walk']/tot_jobs
_df['% Total Jobs (3-mile bike)'] = _df['Jobs within 3-mile Bike']/tot_jobs
_df.to_csv(os.path.join(output_dir,str(incr)+'_walk_bike_jobs_region.csv')); incr+=1
output = _df.to_html(formatters={
    '% Total Jobs (1-mile walk)': '{:0,.1%}'.format,
    '% Total Jobs (3-mile bike)': '{:0,.1%}'.format,
})
display(HTML(output))

,Jobs within 1-mile Walk,Jobs within 3-mile Bike,% Total Jobs (1-mile walk),% Total Jobs (3-mile bike)
geography_value,,,,
King,"15,719","79,153",0.8%,3.9%
Kitsap,"1,225","7,582",0.1%,0.4%
Pierce,"2,189","16,108",0.1%,0.8%
Snohomish,"2,008","17,449",0.1%,0.9%


### Regional Growth Center

In [41]:
_df = df[df['geography_group'] == 'GrowthCenterName']
_df.index = _df.geography_value
_df = _df[['jobs_1_mile_walk','jobs_3_mile_bike']]
_df.rename(columns={'jobs_1_mile_walk': 'Jobs within 1-mile Walk',
                   'jobs_3_mile_bike': 'Jobs within 3-mile Bike'}, inplace=True)
_df.to_csv(os.path.join(output_dir,str(incr)+'_walk_bike_jobs_rgc.csv')); incr+=1
_df['% Total Jobs (1-mile walk)'] = _df['Jobs within 1-mile Walk']/tot_jobs
_df['% Total Jobs (3-mile bike)'] = _df['Jobs within 3-mile Bike']/tot_jobs
_df.to_csv(os.path.join(output_dir,str(incr)+'_walk_bike_jobs_region.csv')); incr+=1
output = _df.to_html(formatters={
    '% Total Jobs (1-mile walk)': '{:0,.1%}'.format,
    '% Total Jobs (3-mile bike)': '{:0,.1%}'.format,
})
display(HTML(output))

,Jobs within 1-mile Walk,Jobs within 3-mile Bike,% Total Jobs (1-mile walk),% Total Jobs (3-mile bike)
geography_value,,,,
Auburn,"11,555","41,672",0.6%,2.1%
Ballard-Interbay,"13,283","80,965",0.7%,4.0%
Bellevue,"52,917","105,281",2.6%,5.2%
Bothell Canyon Park,"7,779","19,229",0.4%,1.0%
Bremerton,"12,155","34,524",0.6%,1.7%
Burien,"4,355","12,877",0.2%,0.6%
Cascade,"1,670","8,580",0.1%,0.4%
Duwamish,"28,832","248,710",1.4%,12.3%
Everett,"16,244","39,883",0.8%,2.0%


### Regional Geography

In [42]:
_df = df[df['geography_group'] == 'rg_proposed']
_df.index = _df.geography_value
_df = _df[['jobs_1_mile_walk','jobs_3_mile_bike']]
_df.rename(columns={'jobs_1_mile_walk': 'Jobs within 1-mile Walk',
                   'jobs_3_mile_bike': 'Jobs within 3-mile Bike'}, inplace=True)
_df.to_csv(os.path.join(output_dir,str(incr)+'_walk_bike_jobs_reg_geog.csv')); incr+=1
_df['% Total Jobs (1-mile walk)'] = _df['Jobs within 1-mile Walk']/tot_jobs
_df['% Total Jobs (3-mile bike)'] = _df['Jobs within 3-mile Bike']/tot_jobs
_df.to_csv(os.path.join(output_dir,str(incr)+'_walk_bike_jobs_region.csv')); incr+=1
output = _df.to_html(formatters={
    '% Total Jobs (1-mile walk)': '{:0,.1%}'.format,
    '% Total Jobs (3-mile bike)': '{:0,.1%}'.format,
})
display(HTML(output))

,Jobs within 1-mile Walk,Jobs within 3-mile Bike,% Total Jobs (1-mile walk),% Total Jobs (3-mile bike)
geography_value,,,,
CitiesTowns,875,"7,189",0.0%,0.4%
Core,"2,820","26,388",0.1%,1.3%
HCT,"1,289","14,079",0.1%,0.7%
Metro,"25,344","120,688",1.3%,6.0%
Rural,106,"1,710",0.0%,0.1%
UU,349,"5,806",0.0%,0.3%


### Equity Geographies

In [43]:
result_df = pd.DataFrame()
for col in geo_list:
    _df = df[(df['geography_group'] == col) & (df['geography_value'] == '1')]
    result_df = result_df.append(_df[['geography_group', 'jobs_1_mile_walk','jobs_3_mile_bike']])
result_df.to_csv(os.path.join(output_dir,str(incr)+'_walk_bike_jobs_equity.csv')); incr+=1
result_df.rename(columns={'geography_group': 'Equity Group', 
                          'jobs_1_mile_walk': 'Jobs within 1-mile Walk',
                        'jobs_3_mile_bike':  'Jobs within 3-mile Bike'}, inplace=True)

result_df['% Total Jobs (1-mile walk)'] = result_df['Jobs within 1-mile Walk']/tot_jobs
result_df['% Total Jobs (3-mile bike)'] = result_df['Jobs within 3-mile Bike']/tot_jobs
output = result_df.to_html(formatters={
    '% Total Jobs (1-mile walk)': '{:0,.1%}'.format,
    '% Total Jobs (3-mile bike)': '{:0,.1%}'.format
}, index=False)
display(HTML(output))

Equity Group,Jobs within 1-mile Walk,Jobs within 3-mile Bike,% Total Jobs (1-mile walk),% Total Jobs (3-mile bike)
youth_geog_vs_reg_total,"1,616","19,287",0.1%,1.0%
elderly_geog_vs_reg_total,"6,726","35,147",0.3%,1.7%
english_geog_vs_reg_total,"6,150","41,066",0.3%,2.0%
racial_geog_vs_reg_total,"13,634","60,378",0.7%,3.0%
racial_geog_vs_50_percent,"6,851","49,358",0.3%,2.4%
poverty_geog_vs_reg_total,"11,676","54,120",0.6%,2.7%
poverty_geog_vs_50_percent,"16,335","57,509",0.8%,2.9%
disability_geog_vs_reg_total,"11,188","41,536",0.6%,2.1%


In [44]:
result_df = pd.DataFrame()
for col in geo_list:
    _df = df[(df['geography_group'] == col) & (df['geography_value'] == '0')]
    result_df = result_df.append(_df[['geography_group', 'jobs_1_mile_walk','jobs_3_mile_bike']])
result_df.to_csv(os.path.join(output_dir,str(incr)+'_walk_bike_jobs_equity.csv')); incr+=1
result_df.rename(columns={'geography_group': 'Equity Group', 
                          'jobs_1_mile_walk': 'Jobs within 1-mile Walk',
                        'jobs_3_mile_bike':  'Jobs within 3-mile Bike'}, inplace=True)

result_df['% Total Jobs (1-mile walk)'] = result_df['Jobs within 1-mile Walk']/tot_jobs
result_df['% Total Jobs (3-mile bike)'] = result_df['Jobs within 3-mile Bike']/tot_jobs
result_df['Equity Group'] = ['NOT in '+i for i in result_df['Equity Group']]
output = result_df.to_html(formatters={
    '% Total Jobs (1-mile walk)': '{:0,.1%}'.format,
    '% Total Jobs (3-mile bike)': '{:0,.1%}'.format
}, index=False)
display(HTML(output))

Equity Group,Jobs within 1-mile Walk,Jobs within 3-mile Bike,% Total Jobs (1-mile walk),% Total Jobs (3-mile bike)
NOT in youth_geog_vs_reg_total,"17,350","81,292",0.9%,4.0%
NOT in elderly_geog_vs_reg_total,"12,074","64,325",0.6%,3.2%
NOT in english_geog_vs_reg_total,"11,578","56,215",0.6%,2.8%
NOT in racial_geog_vs_reg_total,"6,244","42,604",0.3%,2.1%
NOT in racial_geog_vs_50_percent,"10,235","50,861",0.5%,2.5%
NOT in poverty_geog_vs_reg_total,"8,009","47,972",0.4%,2.4%
NOT in poverty_geog_vs_50_percent,"9,386","50,389",0.5%,2.5%
NOT in disability_geog_vs_reg_total,"8,130","58,388",0.4%,2.9%
